## Installing the necessary libraries

In [32]:
!pip install openai
!pip install langchain-openai

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [46]:
pip install langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [48]:
!pip install faiss-cpu

Defaulting to user installation because normal site-packages is not writeable


In [53]:
!pip install langchain

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: C:\ProgramData\anaconda3\python.exe -m pip install --upgrade pip


In [21]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Document loading & splitting
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LLMs
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnables (LangChain Expression Language)
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
# For pretty printing
from textwrap import indent
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

# Document loading & splitting
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# LLMs
from langchain_core.language_models import BaseChatModel
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Runnables (LangChain Expression Language)
from langchain_core.runnables import RunnablePassthrough, RunnableParallel

# For pretty printing
from textwrap import indent

In [22]:
#1: Load API key from .env file
from dotenv import load_dotenv
import os

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [24]:
#2: Load the document
with open("state_of_the_union.txt","r", encoding="utf8") as f:
  data = f.read()
  
#3: Use Text Loader to load the document
loder=TextLoader('state_of_the_union.txt', encoding="utf8")
document=loder.load()
print(document[0].page_content)

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

Groups of citizens blocking tanks with their bodies. Every

In [25]:
# 4: Split the document into chunks
text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=50)
text_chunks=text_splitter.split_documents(document)

In [26]:
#5: Print a sample text chunk
print(text_chunks[3].page_content)

Please rise if you are able and show that, Yes, we the United States of America stand with the Ukrainian people. 

Throughout our history we’ve learned this lesson when dictators do not pay a price for their aggression they cause more chaos.   

They keep moving.   

And the costs and the threats to America and the world keep rising.   

That’s why the NATO Alliance was created to secure peace and stability in Europe after World War 2.


In [27]:
#6: Create FAISS vector store
embeddings=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectorstore=FAISS.from_documents(text_chunks, embeddings)
retriever=vectorstore.as_retriever()

In [ ]:
7

In [28]:
#7 System prompt template
template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [29]:
#8: Initialize prompt, output parser, and LLM model
prompt=ChatPromptTemplate.from_template(template)
output_parser=StrOutputParser()
llm_model=ChatOpenAI(openai_api_key=OPENAI_API_KEY,model_name="gpt-3.5-turbo")

In [ ]:
#9: Create RAG chain
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | llm_model
    | output_parser
)

In [30]:
#10: Test RAG chain
rag_chain.invoke("How is the United States supporting Ukraine economically and militarily?")

'The United States is supporting Ukraine economically and militarily by providing military assistance, economic assistance, and humanitarian assistance. They are giving more than $1 billion in direct assistance to Ukraine. The U.S. will continue to aid the Ukrainian people in their fight for freedom and to help ease their suffering. Despite not engaging in conflict with Russian forces in Ukraine, the U.S. stands with the Ukrainian people. The world is choosing the side of peace and security in the battle between democracy and autocracy. The NATO Alliance was created to secure peace and stability in Europe after World War 2. President Zelenskyy said, "Light will win over darkness" in his speech to the European Parliament. The Ukrainian Ambassador to the United States is present to receive support. Citizens of Ukraine, from students to retirees, are defending their homeland against aggression.'

In [31]:
rag_chain.invoke("What is capital of India?")

"I don't know."